# Síťová komunikace v AsyncIO (Streams)

Tento notebook demonstruje použití asynchronních streamů pro komunikaci klient-server. 

V Jupyteru je toto trochu složitější, protože server musí běžet na pozadí, zatímco klient (v jiné buňce nebo stejné) posílá data.

## 1. Definice asynchronního serveru

In [ ]:
import asyncio

async def handle_client(reader, writer):
    addr = writer.get_extra_info('peername')
    print(f"[Server] Klient připojen: {addr}")

    data = await reader.read(100)
    message = data.decode()
    
    print(f"[Server] Přijato {message!r} od {addr}")

    # Echo zpět
    print(f"[Server] Posílám echo zpět klientovi {addr}...")
    writer.write(data)
    await writer.drain()

    print(f"[Server] Zavírám spojení s {addr}")
    writer.close()
    await writer.wait_closed()

async def run_server():
    # Spustíme server na portu 8888
    server = await asyncio.start_server(handle_client, '127.0.0.1', 8888)
    
    async with server:
        # Nastavíme timeout, aby notebook nezůstal viset navždy
        try:
            print("[Server] Startuji a naslouchám na 127.0.0.1:8888...")
            await asyncio.wait_for(server.serve_forever(), timeout=5.0)
        except asyncio.TimeoutError:
            print("[Server] 5 sekund vypršelo, server se automaticky vypíná.")

## 2. Definice asynchronního klienta

In [ ]:
async def run_client(message):
    print(f'[Klient] Připojuji se k serveru...')
    try:
        reader, writer = await asyncio.open_connection('127.0.0.1', 8888)

        print(f'[Klient] Posílám: {message!r}')
        writer.write(message.encode())
        await writer.drain()

        data = await reader.read(100)
        print(f'[Klient] Přijato echo od serveru: {data.decode()!r}')

        print('[Klient] Úloha hotova, zavírám connection.')
        writer.close()
        await writer.wait_closed()
    except Exception as e:
        print(f'[Klient] CHYBA: {e}')

## 3. Spuštění simulace

V asynchronním programování je důležité, aby server běžel dříve, než se klient pokusí připojit. V notebooku to zajistíme tak, že server spustíme jako úlohu na pozadí (`create_task`) a chvilku počkáme.

In [ ]:
async def main():
    print("--- START SIMULACE ---")
    
    # 1. Spustíme server jako task na pozadí
    server_task = asyncio.create_task(run_server())
    
    # 2. Krátce počkáme (neblokující!), aby server stihl nastartovat a provést bind
    await asyncio.sleep(1)
    
    # 3. Spustíme klienty souběžně a počkáme na jejich dokončení
    print("[Main] Spouštím klienty...")
    await asyncio.gather(
        run_client('Ahoj 1!'),
        run_client('Ahoj 2!'),
        return_exceptions=True
    )
    
    # 4. Počkáme na dokončení serveru (v našem případě na vypršení 5s timeoutu)
    print("[Main] Klienti dokončeni, čekám na konec serveru...")
    await server_task
    print("--- KONEC SIMULACE ---")

await main()

--- START SIMULACE ---
[Server] Startuji a naslouchám na 127.0.0.1:8888...
[Main] Spouštím klienty...
[Klient] Připojuji se k serveru...
[Klient] Připojuji se k serveru...
[Server] Klient připojen: ('127.0.0.1', 59416)
[Server] Klient připojen: ('127.0.0.1', 59430)
[Klient] Posílám: 'Ahoj 1!'
[Klient] Posílám: 'Ahoj 2!'
[Server] Přijato 'Ahoj 1!' od ('127.0.0.1', 59416)
[Server] Posílám echo zpět klientovi ('127.0.0.1', 59416)...
[Server] Zavírám spojení s ('127.0.0.1', 59416)
[Server] Přijato 'Ahoj 2!' od ('127.0.0.1', 59430)
[Server] Posílám echo zpět klientovi ('127.0.0.1', 59430)...
[Server] Zavírám spojení s ('127.0.0.1', 59430)
[Klient] Přijato echo od serveru: 'Ahoj 1!'
[Klient] Úloha hotova, zavírám connection.
[Klient] Přijato echo od serveru: 'Ahoj 2!'
[Klient] Úloha hotova, zavírám connection.
[Main] Klienti dokončeni, čekám na konec serveru...
[Server] 5 sekund vypršelo, server se automaticky vypíná.
--- KONEC SIMULACE ---


## 4. Úkol pro studenty

Upravte `handle_client` tak, aby:
1. Nejdříve poslal klientovi uvítací zprávu "WELCOME".
2. Poté čekal na jméno klienta.
3. Odpověděl "AHOJ [jmeno]".
4. Upravte i klienta, aby s tímto novým protokolem uměl komunikovat.